In [38]:
import pandas as pd
import numpy as np
from IPython.display import display

### <font color="orange">データの読み込み</font>

In [39]:
import zipfile
import os
# 解凍したいzipファイルのパス
zip_file_path = '../../../../0_original_datasets/datasets.zip'

# 解凍先のディレクトリ
folder_path = '../../../tmp_data_folder/0_tmp_original_data/'

if os.path.exists(folder_path):
    print("既に解凍されたフォルダが存在します。")
    pass
else:
    # Zipファイルを開いて解凍する
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(folder_path)

    print(f'{zip_file_path} を {folder_path} に解凍しました。')

../../../../0_original_datasets/datasets.zip を ../../../tmp_data_folder/0_tmp_original_data/ に解凍しました。


In [40]:
olist_order_payments_dataset = pd.read_csv("../../../tmp_data_folder/0_tmp_original_data/datasets/olist_order_payments_dataset.csv")
olist_order_items_dataset = pd.read_csv("../../../tmp_data_folder/0_tmp_original_data/datasets/olist_order_items_dataset.csv")


# payment_type=="not_defined"は支払いが無いので、行を削除する。
display(olist_order_payments_dataset[olist_order_payments_dataset["payment_type"]=="not_defined"])
olist_order_payments_dataset = olist_order_payments_dataset.drop(olist_order_payments_dataset[olist_order_payments_dataset["payment_type"]=="not_defined"].index)
display(olist_order_payments_dataset[olist_order_payments_dataset["payment_type"]=="not_defined"])

,order_id,payment_sequential,payment_type,payment_installments,payment_value
51280,4637ca194b6387e2d538dc89b124b0ee,1,not_defined,1,0.0
57411,00b1cb0320190ca0daa2c88b35206009,1,not_defined,1,0.0
94427,c8c528189310eaa44a745b8d9d26908b,1,not_defined,1,0.0


,order_id,payment_sequential,payment_type,payment_installments,payment_value


### <font color="orange">データの確認</font>

In [41]:
print(olist_order_payments_dataset[olist_order_payments_dataset.duplicated(subset="order_id", keep=False)].shape[0])
print(olist_order_items_dataset[olist_order_items_dataset.duplicated(subset="order_id", keep=False)].shape[0])

#* ともに重複有り。

7407
23787


In [42]:
# "olist_order_payment_dataset"  と  "olist_order_item_dataset"の両方で重複する行のみを抽出する。

# olist_order_payment_datasetで重複するorder_idを抽出する。
payment_duplication_order_id = olist_order_payments_dataset[olist_order_payments_dataset.duplicated(subset="order_id", keep="first")].order_id.values

# olist_order_item_datasetで重複するorder_idを抽出する。
item_duplication_order_id = olist_order_items_dataset[olist_order_items_dataset.duplicated(subset="order_id", keep="first")].order_id.values

# payment_duplication_order_id とitem_duplication_order_idで一致するorder_idのみ抽出する。
final_order_id = set(payment_duplication_order_id) & set(item_duplication_order_id)
final_order_id = list(final_order_id)

# データを確認する。
id = final_order_id[0]
display(olist_order_payments_dataset.query("order_id == @id"))
display(olist_order_items_dataset.query("order_id == @id"))

,order_id,payment_sequential,payment_type,payment_installments,payment_value
37453,f7dc79358f6495f086b1874bae7a9575,2,voucher,1,100.00
96952,f7dc79358f6495f086b1874bae7a9575,1,credit_card,2,138.15


,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
109054,f7dc79358f6495f086b1874bae7a9575,1,e74d829aead9632c343d367969aec904,0ea22c1cfbdc755f86b9b54b39c16043,2017-12-12 20:37:32,59.9,4.05
109055,f7dc79358f6495f086b1874bae7a9575,2,94f2d239e688aa415a2dd620435e8344,bb5bd94c8f59cc050072d2930c869cac,2017-12-12 20:37:32,149.9,24.30


In [43]:
# order_idごとの
# "olist_order_payments_dataset"の "payment_value"の合計と、
# "olist_order_items_dataset" の "price"と"freight_value"の合計が一致しているように見えたので確認する。

tmp_payment = olist_order_payments_dataset.groupby("order_id", as_index=False)["payment_value"].sum()
tmp_payment["payment_value"] = tmp_payment["payment_value"].round(2)    #浮動小数点の計算をするため。np.isclose()というものもある。

tmp_item = olist_order_items_dataset.groupby("order_id", as_index=False)[["price", "freight_value"]].sum()
tmp_item["total_item_value"] = tmp_item["price"] + tmp_item["freight_value"]
tmp_item["total_item_value"] = tmp_item["total_item_value"].round(2)    #浮動小数点の計算をするため。np.isclose()というものもある。
tmp_item = tmp_item[["order_id", "total_item_value"]]

check_df = pd.merge(tmp_payment, tmp_item, on="order_id", how="inner")

check_df["diff"] = check_df["payment_value"] - check_df["total_item_value"].round(2)

print(f"一致 　: {len(check_df[check_df['diff'] == 0.0].sort_values(by='diff', ascending=False))}件")
print(f"不一致 : {len(check_df[check_df['diff'] != 0.0].sort_values(by='diff', ascending=False))}件")

print("="*20 + " 不一致データ一覧 " + "="*20)
check_df[check_df["diff"] != 0.0].sort_values(by="diff", ascending=False)

#* 支払金額の不一致が576件あった。
#* 分割払いの回数が多いデータなどはpayment_valueの方が大きい傾向にあるようだが、詳細な要因は不明。

一致 　: 98089件
不一致 : 576件
==================== 不一致データ一覧 ====================


,order_id,payment_value,total_item_value,diff
79537,ce6d150fb29ada17d2082f4847107665,1586.47,1403.66,182.81
42515,6e5fe7366a2e1bfbf3257dba0af1267f,406.92,287.91,119.01
43434,70b742795bc441e94a44a084b6d9ce7a,578.82,466.93,111.89
58752,996c7e73600ad3723e8627ab7bef81e4,664.43,587.90,76.53
43437,70b7e94ea46d3e8b5bc12a50186edaf0,274.84,213.15,61.69
...,...,...,...,...
42292,6dcf0aeb8b1eb4021c26e1d0e9394979,318.97,333.92,-14.95
25102,4154bf1348caac78152fe76e3e9c4af8,150.27,165.26,-14.99
42504,6e57e23ecac1ae881286657694444267,333.91,350.41,-16.50
97605,fd33085945f15975375cd8ec85440511,212.82,234.62,-21.80


### <font color="orange">データ作成"itemss_dataset"について,order_idが一意になるように整形する。</font>

In [44]:
# itemss_datasetについて
id = final_order_id[0]
display(olist_order_items_dataset.query("order_id == @id"))

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
109054,f7dc79358f6495f086b1874bae7a9575,1,e74d829aead9632c343d367969aec904,0ea22c1cfbdc755f86b9b54b39c16043,2017-12-12 20:37:32,59.9,4.05
109055,f7dc79358f6495f086b1874bae7a9575,2,94f2d239e688aa415a2dd620435e8344,bb5bd94c8f59cc050072d2930c869cac,2017-12-12 20:37:32,149.9,24.30


In [45]:
# order_id, product_id, seller_idが重複する行は削除する。
custom_items_dataset = olist_order_items_dataset.drop_duplicates(subset=["order_id", "product_id", "seller_id"])
custom_items_dataset = custom_items_dataset[["order_id", "seller_id", "product_id"]]

In [46]:
# olist_products_dataset, olist_category_name_transrationをマージする。
olist_products_dataset = pd.read_csv("../../../tmp_data_folder/0_tmp_original_data/datasets/olist_products_dataset.csv")
olist_category_name_transration = pd.read_csv("../../../tmp_data_folder/0_tmp_original_data/datasets/product_category_name_translation.csv")

custom_items_dataset = pd.merge(custom_items_dataset, olist_products_dataset, on="product_id", how="left")
custom_items_dataset = pd.merge(custom_items_dataset, olist_category_name_transration, on="product_category_name", how="left")

In [47]:
# product_length_cm, product_height_cm, product_width_cmから体積を算出する。
custom_items_dataset["product_volume"] = custom_items_dataset["product_length_cm"]*custom_items_dataset["product_height_cm"]*custom_items_dataset["product_width_cm"]

In [48]:
custom_items_dataset = custom_items_dataset[[
    "order_id",
    "seller_id",
    "product_category_name_english",
    "product_name_lenght",
    "product_description_lenght",
    "product_photos_qty",
    "product_weight_g",
    "product_volume"  
]]

In [49]:
custom_items_dataset.drop_duplicates(inplace=True)

In [50]:
custom_items_dataset.head()

,order_id,seller_id,product_category_name_english,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_volume
0,00010242fe8c5a6d1ba2dd792cb16214,48436dade18ac8b2bce089ec2a041202,cool_stuff,58.0,598.0,4.0,650.0,3528.0
1,00018f77f2f0320c557190d7a144bdd3,dd7ddc04e1b6c2c614352b383efe2d36,pet_shop,56.0,239.0,2.0,30000.0,60000.0
2,000229ec398224ef6ca0657da4fc703e,5b51032eddd242adc84c38acab88f23d,furniture_decor,59.0,695.0,2.0,3050.0,14157.0
3,00024acbcdf0a6daa1e931b038114c75,9d7a1d34a5052409006425275ba1c2b4,perfumery,42.0,480.0,1.0,200.0,2400.0
4,00042b26cf59d7ce69dfabb4e55b4fd9,df560393f3a51e74553ab94004ba5c87,garden_tools,59.0,409.0,1.0,3750.0,42000.0


### <font color="orange">データの保存</font>

In [51]:
import os

save_path = "../../../tmp_data_folder/1_custom_data/"

# フォルダが存在しない場合に作成
if not os.path.exists(save_path):
    os.makedirs(save_path)
    print(f'フォルダ {save_path} を作成しました。')
else:
    print(f'フォルダ {save_path} は既に存在します。')
    
custom_items_dataset.to_csv("../../../tmp_data_folder/1_custom_data/custom_items_dataset.csv", index=False)
print("../tmp/にunique_olist_review_dataset.csvを保存しました。")

フォルダ ../../../tmp_data_folder/1_custom_data/ を作成しました。
../tmp/にunique_olist_review_dataset.csvを保存しました。
